In [15]:
import numpy as np
import pandas as pd
from nsepy import get_history
import os
import matplotlib.pyplot as plt
import datetime as dt

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [16]:
start=dt.date(2015,1,1)
end=dt.date(2023,1,5)

In [17]:
data = get_history(symbol="SBIN",start=start,end=end)
data

,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
Date,,,,,,,,,,,,,,
2015-01-01,SBIN,EQ,311.85,312.45,315.00,310.70,314.00,314.00,313.67,6138488,1.925489e+14,58688,1877677,0.3059
2015-01-02,SBIN,EQ,314.00,314.35,318.30,314.35,315.60,315.25,316.80,9935094,3.147389e+14,79553,4221685,0.4249
2015-01-05,SBIN,EQ,315.25,316.25,316.80,312.10,312.80,312.75,313.84,9136716,2.867432e+14,88236,3845173,0.4208
2015-01-06,SBIN,EQ,312.75,310.00,311.10,298.70,299.90,299.90,305.14,15329257,4.677601e+14,169268,7424847,0.4844
2015-01-07,SBIN,EQ,299.90,300.00,302.55,295.15,301.40,300.15,299.95,15046745,4.513243e+14,147185,5631400,0.3743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-30,SBIN,EQ,611.80,615.25,620.55,611.35,614.25,613.70,615.43,13052895,8.033100e+14,158347,6160516,0.4720
2023-01-02,SBIN,EQ,613.70,614.95,619.20,611.20,612.60,612.20,614.42,6726084,4.132615e+14,125418,2430000,0.3613
2023-01-03,SBIN,EQ,612.20,614.50,616.60,610.00,612.95,612.40,613.26,6818062,4.181258e+14,122799,2451217,0.3595


In [18]:
data.drop(columns = ["Symbol","Series","Prev Close","VWAP","Turnover","%Deliverble"],inplace=True)

In [19]:
data

,Open,High,Low,Last,Close,Volume,Trades,Deliverable Volume
Date,,,,,,,,
2015-01-01,312.45,315.00,310.70,314.00,314.00,6138488,58688,1877677
2015-01-02,314.35,318.30,314.35,315.60,315.25,9935094,79553,4221685
2015-01-05,316.25,316.80,312.10,312.80,312.75,9136716,88236,3845173
2015-01-06,310.00,311.10,298.70,299.90,299.90,15329257,169268,7424847
2015-01-07,300.00,302.55,295.15,301.40,300.15,15046745,147185,5631400
...,...,...,...,...,...,...,...,...
2022-12-30,615.25,620.55,611.35,614.25,613.70,13052895,158347,6160516
2023-01-02,614.95,619.20,611.20,612.60,612.20,6726084,125418,2430000
2023-01-03,614.50,616.60,610.00,612.95,612.40,6818062,122799,2451217


In [23]:
data["Returns"] = ((data["Close"]-data["Open"])/data["Open"])*100
data

,Open,High,Low,Last,Close,Volume,Trades,Deliverable Volume,Returns
Date,,,,,,,,,
2015-01-01,312.45,315.00,310.70,314.00,314.00,6138488,58688,1877677,0.496079
2015-01-02,314.35,318.30,314.35,315.60,315.25,9935094,79553,4221685,0.286305
2015-01-05,316.25,316.80,312.10,312.80,312.75,9136716,88236,3845173,-1.106719
2015-01-06,310.00,311.10,298.70,299.90,299.90,15329257,169268,7424847,-3.258065
2015-01-07,300.00,302.55,295.15,301.40,300.15,15046745,147185,5631400,0.050000
...,...,...,...,...,...,...,...,...,...
2022-12-30,615.25,620.55,611.35,614.25,613.70,13052895,158347,6160516,-0.251930
2023-01-02,614.95,619.20,611.20,612.60,612.20,6726084,125418,2430000,-0.447191
2023-01-03,614.50,616.60,610.00,612.95,612.40,6818062,122799,2451217,-0.341741


In [34]:
x = data.copy()
x.drop(columns = ["Close"],inplace=True)
x = x.values
y = data.iloc[:,4].values

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state = 0,test_size=0.25)

In [35]:
def LSTM_model():
    
    model = Sequential()
    
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1],1)))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))

    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    
    model.add(Dense(units=1))
    
    return model

In [36]:
model = LSTM_model()
model.summary()
model.compile(optimizer='adam', 
              loss='mean_squared_error',metrics = ["accuracy"])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 8, 50)             10400     
                                                                 
 dropout_9 (Dropout)         (None, 8, 50)             0         
                                                                 
 lstm_10 (LSTM)              (None, 8, 50)             20200     
                                                                 
 dropout_10 (Dropout)        (None, 8, 50)             0         
                                                                 
 lstm_11 (LSTM)              (None, 50)                20200     
                                                                 
 dropout_11 (Dropout)        (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                

In [37]:
checkpointer = ModelCheckpoint(filepath = 'weights_best.hdf5', 
                               verbose = 2, 
                               save_best_only = True)

model.fit(x_train, 
          y_train, 
          epochs=25, 
          batch_size = 32,
          callbacks = [checkpointer])

Epoch 1/25
47/47 [==============================] - 5s 12ms/step - loss: 107245.2500 - accuracy: 0.0000e+00
Epoch 2/25
47/47 [==============================] - 1s 12ms/step - loss: 103015.7344 - accuracy: 0.0000e+00
Epoch 3/25
47/47 [==============================] - 1s 11ms/step - loss: 101233.8125 - accuracy: 0.0000e+00
Epoch 4/25
47/47 [==============================] - 1s 12ms/step - loss: 99677.9688 - accuracy: 0.0000e+00
Epoch 5/25
47/47 [==============================] - 1s 12ms/step - loss: 98163.7969 - accuracy: 0.0000e+00
Epoch 6/25
47/47 [==============================] - 1s 13ms/step - loss: 96697.3281 - accuracy: 0.0000e+00
Epoch 7/25
47/47 [==============================] - 1s 12ms/step - loss: 95243.2578 - accuracy: 0.0000e+00
Epoch 8/25
47/47 [==============================] - 1s 12ms/step - loss: 93841.8984 - accuracy: 0.0000e+00
Epoch 9/25
47/47 [==============================] - 1s 12ms/step - loss: 92480.8281 - accuracy: 0.0000e+00
Epoch 10/25
47/47 [===============